# Fileupload

In [1]:
import arviz as az
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pymc as pm
import plotly.express as px

print(f"Running on PyMC v{pm.__version__}")

<frozen importlib._bootstrap>:241: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


Running on PyMC v4.2.2


In [2]:
import stan
import nest_asyncio
nest_asyncio.apply()


In [3]:
data = pd.read_csv("./logremoval/testdata.csv")

In [27]:
inflow = data["inflow"].dropna().astype('int32')

In [28]:
inflow

0      345678
1       89905
2      567878
3       56789
4       52535
5        8765
7     2345678
8      987654
9       54322
10       9865
11       5432
12     987654
Name: inflow, dtype: int32

In [29]:
standata = {"Y": inflow.values.tolist(),
           "N": len(data["inflow"].dropna().values),
           "N_new":1}

In [30]:
standata

{'Y': [345678,
  89905,
  567878,
  56789,
  52535,
  8765,
  2345678,
  987654,
  54322,
  9865,
  5432,
  987654],
 'N': 12,
 'N_new': 1}

In [31]:
import stan

schools_code = """
data {
  int<lower=1> N;  // total number of observations
  int Y[N];  // response variable
  int N_new;
 
}
transformed data {
}
parameters {
  real Intercept;  // temporary intercept for centered predictors
  real<lower=0> shape;  // shape parameter
}
transformed parameters {
}
model {
  // likelihood including constants
 
  // initialize linear predictor term
  vector[N] mu = Intercept + rep_vector(0.0, N);
  target += neg_binomial_2_log_lpmf(Y | mu, shape);

  // priors including constants
  target += student_t_lpdf(Intercept | 3, 11.2, 3);
  target += gamma_lpdf(shape | 0.01, 0.01);
}
generated quantities {
  // actual population-level intercept
  real b_Intercept = Intercept;
  vector[N_new] y_sim;
  
  for (n in 1:N_new) {
    y_sim[n] = neg_binomial_2_rng(exp(b_Intercept), shape);
  }
  
}

"""




In [32]:
posterior = stan.build(schools_code, data=standata)

Building: 0.3s
Building: 0.4s
Building: 0.5s
Building: 0.6s
Building: 0.8s
Building: 0.9s
Building: 1.0s
Building: 1.1s
Building: 1.2s
Building: 1.3s
Building: 1.4s
Building: 1.5s
Building: 1.6s
Building: 1.7s
Building: 1.8s
Building: 1.9s
Building: 2.1s
Building: 2.2s
Building: 2.3s
Building: 2.4s
Building: 2.5s
Building: 2.6s
Building: 2.7s
Building: 2.8s
Building: 2.9s
Building: 3.0s
Building: 3.2s
Building: 3.3s
Building: 3.4s
Building: 3.5s
Building: 3.6s
Building: 3.7s
Building: 3.8s
Building: 3.9s
Building: 4.0s
Building: 4.1s
Building: 4.2s
Building: 4.4s
Building: 4.5s
Building: 4.6s
Building: 4.7s
Building: 4.8s
Building: 4.9s
Building: 5.0s
Building: 5.1s
Building: 5.2s
Building: 5.3s
Building: 5.5s
Building: 5.6s
Building: 5.7s
Building: 5.8s
Building: 5.9s
Building: 6.0s
Building: 6.1s
Building: 6.2s
Building: 6.3s
Building: 6.4s
Building: 6.5s
Building: 6.7s
Building: 6.8s
Building: 6.9s
Building: 7.0s
Building: 7.1s
Building: 7.2s
Building: 7.3s
Building: 7.4s
Building: 

In file included from /home/wseis/miniconda3/envs/qmra/lib/python3.10/site-packages/httpstan/include/stan/model/indexing.hpp:5,
                 from /home/wseis/miniconda3/envs/qmra/lib/python3.10/site-packages/httpstan/include/stan/model/model_header.hpp:17,
                 from /home/wseis/.cache/httpstan/4.8.2/models/oggp5qeg/model_oggp5qeg.cpp:2:
/home/wseis/miniconda3/envs/qmra/lib/python3.10/site-packages/httpstan/include/stan/model/indexing/assign_varmat.hpp: In function 'void stan::model::assign(Mat1&&, const Mat2&, const char*, const stan::model::index_multi&, const stan::model::index_multi&)':
/home/wseis/miniconda3/envs/qmra/lib/python3.10/site-packages/httpstan/include/stan/model/indexing/assign_varmat.hpp:401:9: warning: typedef 'using pair_type = struct std::pair<int, std::vector<int, stan::math::arena_allocator<int> > >' locally defined but not used [-Wunused-local-typedefs]
  401 |   using pair_type = std::pair<int, arena_vec>;
      |         ^~~~~~~~~


Building: 24.6s
Building: 24.7s


/home/wseis/.cache/httpstan/4.8.2/models/oggp5qeg/model_oggp5qeg.cpp: In constructor 'model_oggp5qeg_namespace::model_oggp5qeg::model_oggp5qeg(stan::io::var_context&, unsigned int, std::ostream*)':
/home/wseis/.cache/httpstan/4.8.2/models/oggp5qeg/model_oggp5qeg.cpp:64:11: warning: variable 'pos__' set but not used [-Wunused-but-set-variable]
   64 |       int pos__ = std::numeric_limits<int>::min();
      |           ^~~~~


Building: 24.9s
Building: 25.0s
Building: 25.1s
Building: 25.2s
Building: 25.3s
Building: 25.5s
Building: 25.6s
Building: 25.7s
Building: 25.8s
Building: 25.9s
Building: 26.1s
Building: 26.2s
Building: 26.3s
Building: 26.4s
Building: 26.5s
Building: 26.7s
Building: 26.8s
Building: 26.9s
Building: 27.0s


/home/wseis/.cache/httpstan/4.8.2/models/oggp5qeg/model_oggp5qeg.cpp: In instantiation of 'void model_oggp5qeg_namespace::model_oggp5qeg::transform_inits_impl(VecVar&, VecI&, VecVar&, std::ostream*) const [with VecVar = std::vector<double, std::allocator<double> >; VecI = std::vector<int>; stan::require_vector_t<T_y>* <anonymous> = 0; stan::require_vector_like_vt<std::is_integral, VecI>* <anonymous> = 0; std::ostream = std::basic_ostream<char>]':
/home/wseis/.cache/httpstan/4.8.2/models/oggp5qeg/model_oggp5qeg.cpp:404:69:   required from here
/home/wseis/.cache/httpstan/4.8.2/models/oggp5qeg/model_oggp5qeg.cpp:229:11: warning: variable 'pos__' set but not used [-Wunused-but-set-variable]
  229 |       int pos__ = std::numeric_limits<int>::min();
      |           ^~~~~


Building: 27.1s
Building: 27.2s
Building: 27.3s
Building: 27.4s
Building: 27.5s
Building: 27.7s
Building: 27.8s
Building: 27.9s
Building: 28.0s
Building: 28.1s
Building: 28.2s
Building: 28.3s
Building: 28.4s
Building: 28.5s
Building: 28.6s
Building: 28.7s
Building: 28.9s
Building: 29.0s
Building: 29.1s
Building: 29.2s
Building: 29.3s
Building: 29.4s
Building: 29.5s
Building: 29.6s
Building: 29.7s
Building: 29.9s
Building: 30.0s
Building: 30.1s
Building: 30.2s
Building: 30.3s
Building: 30.4s
Building: 30.5s
Building: 30.6s
Building: 30.8s
Building: 30.9s
Building: 31.0s
Building: 31.1s
Building: 31.2s
Building: 31.3s
Building: 31.4s
Building: 31.5s
Building: 31.7s
Building: 31.8s
Building: 31.9s
Building: 32.0s
Building: 32.1s
Building: 32.2s
Building: 32.3s
Building: 32.4s
Building: 32.5s
Building: 32.7s
Building: 32.8s
Building: 32.9s
Building: 33.0s
Building: 33.1s
Building: 33.2s
Building: 33.3s
Building: 33.4s
Building: 33.5s
Building: 33.6s
Building: 33.7s
Building: 33.8s
Building

Building: 44.0s, done.
Messages from stanc:
Warning in '/tmp/httpstan_rriutj5w/model_oggp5qeg.stan', line 4, column 2: Declaration
    of arrays by placing brackets after a variable name is deprecated and
    will be removed in Stan 2.32.0. Instead use the array keyword before the
    type. This can be changed automatically using the auto-format flag to
    stanc


In [33]:
fit = posterior.sample(num_chains=4, num_samples=1000)
df = fit.to_frame()  # pandas `DataFrame, requires pandas

Sampling:   0%
Sampling:  25% (2000/8000)
Sampling:  50% (4000/8000)
Sampling:  75% (6000/8000)
Sampling: 100% (8000/8000)
Sampling: 100% (8000/8000), done.
Messages received during sampling:
  Gradient evaluation took 1.1e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.11 seconds.
  Adjust your expectations accordingly!
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_log_lpmf: Precision parameter is 0, but must be positive finite! (in '/tmp/httpstan_c__22z14/model_oggp5qeg.stan', line 21, column 2 to column 51)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issu

  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_log_lpmf: Precision parameter is 0, but must be positive finite! (in '/tmp/httpstan_c__22z14/model_oggp5qeg.stan', line 21, column 2 to column 51)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_log_lpmf: Precision parameter is 0, but must be positive finite! (in '/tmp/httpstan

  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_log_lpmf: Precision parameter is 0, but must be positive finite! (in '/tmp/httpstan_c__22z14/model_oggp5qeg.stan', line 21, column 2 to column 51)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_log_lpmf: Precision parameter is 0, but must be positive finite! (in '/tmp/httpstan

In [35]:
np.quantile(df["y_sim.1"], q = [.1, .5, .9])

array([   2126.4,  178956. , 1419222.6])

In [36]:
df.describe()

parameters,lp__,accept_stat__,stepsize__,treedepth__,n_leapfrog__,divergent__,energy__,Intercept,shape,b_Intercept,y_sim.1
count,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.0,4000.000000,4000.000000,4000.000000,4000.000000,4.000000e+03
mean,-172.889600,0.929776,0.732964,2.136750,4.486000,0.0,173.878069,13.091787,0.428065,13.091787,5.530493e+05
std,1.095796,0.109142,0.118763,0.625817,2.100216,0.0,1.473432,0.487449,0.145428,0.487449,1.066072e+06
min,-180.752833,0.078163,0.589163,1.000000,1.000000,0.0,171.858928,11.326490,0.110083,11.326490,0.000000e+00
25%,-173.291019,0.912559,0.669892,2.000000,3.000000,0.0,172.783461,12.754488,0.324865,12.754488,2.741050e+04
50%,-172.544569,0.968918,0.712145,2.000000,3.000000,0.0,173.562825,13.051969,0.410556,13.051969,1.789560e+05
75%,-172.125946,0.995400,0.775218,3.000000,7.000000,0.0,174.609648,13.359121,0.515567,13.359121,6.413532e+05
max,-171.830732,1.000000,0.918404,3.000000,7.000000,0.0,181.792180,16.015817,1.200678,16.015817,2.125901e+07
